In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv("../../input/optiver-realized-volatility-prediction/x_ref.csv")

In [3]:
train = pd.read_csv("../../input/optiver-realized-volatility-prediction/train.csv")

In [4]:
df_train = data.iloc[:train.shape[0], :]

In [5]:
df_train["target"] = train["target"]

<ipython-input-5-61b04d88f426>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train["target"] = train["target"]


In [6]:
df_train.head()

stock_id  time_id   wap1_sum  wap1_std    wap2_sum  wap2_std  \
0         0        5  303.12506  0.000693  303.105530  0.000781   
1         0       11  200.04778  0.000262  200.041170  0.000272   
2         0       16  187.91385  0.000864  187.939820  0.000862   
3         0       31  119.85978  0.000757  119.835945  0.000656   
4         0       62  175.93286  0.000258  175.934250  0.000317   

   log_return1_realized_volatility  log_return2_realized_volatility  \
0                         0.004499                         0.006999   
1                         0.001204                         0.002476   
2                         0.002369                         0.004801   
3                         0.002574                         0.003637   
4                         0.001894                         0.003257   

   wap_balance_sum  wap_balance_amax  ...  trade_tendency_1_cluster_std  \
0         0.117051          0.001414  ...                      0.915979   
1         0.042309          0.000639  ...                      1.320335   
2         0.062230          0.001135  ...                      0.927733   
3         0.045610          0.001082  ...                      0.819245   
4         0.044783          0.000724  ...                      1.035829   

   trade_tendency_3_cluster_std  trade_tendency_4_cluster_std  \
0                      1.386520                      1.516825   
1                      1.473014                      2.140618   
2                      1.464131                      1.272721   
3                      1.308741                      1.627562   
4                      1.621386                      0.895411   

   trade_tendency_6_cluster_std  trade_energy_0_cluster_std  \
0                      1.880498                    0.002937   
1                      5.496753                    0.001582   
2                      1.637834                    0.001529   
3                      1.676842                    0.001767   
4                      2.131646                    0.001729   

   trade_energy_1_cluster_std  trade_energy_3_cluster_std  \
0                    0.003467                    0.002080   
1                    0.003014                    0.001729   
2                    0.004564                    0.001734   
3                    0.003082                    0.001931   
4                    0.002874                    0.001835   

   trade_energy_4_cluster_std  trade_energy_6_cluster_std    target  
0                    0.001781                    0.003622  0.004136  
1                    0.003176                    0.001635  0.001445  
2                    0.002456                    0.002936  0.002168  
3                    0.003396                    0.002943  0.002195  
4                    0.002135                    0.002500  0.001747  

[5 rows x 514 columns]

In [7]:
def visualize_continuous_feature(df_train, continuous_feature):
            
    print(continuous_feature)

    print("Training Mean: {}  - Training Median: {} - Training Std: {}".format(df_train[continuous_feature].mean(), df_train[continuous_feature].median(), df_train[continuous_feature].std()))
    print("Training Min: {}  - Training Max: {}".format(df_train[continuous_feature].min(), df_train[continuous_feature].max()))
    print("Training Skew: {}  - Training Kurtosis: {}".format(df_train[continuous_feature].skew(), df_train[continuous_feature].kurtosis()))

    fig, axes = plt.subplots(ncols=2, figsize=(24, 6), dpi=100, constrained_layout=True)
    title_size = 18
    label_size = 18

    sns.kdeplot(df_train[continuous_feature], label="Training", fill=True, ax=axes[0])
    axes[0].set_xlabel("")
    axes[0].tick_params(axis="x", labelsize=label_size)
    axes[0].tick_params(axis="y", labelsize=label_size)
    axes[0].legend()
    axes[0].set_title(f"{continuous_feature} Distribution in Training Set", size=title_size, pad=title_size)
    
    sns.scatterplot(x=df_train[continuous_feature], y=df_train["target"], ax=axes[1])
    axes[1].set_title(f"{continuous_feature} vs target", size=title_size, pad=title_size)
    axes[1].set_xlabel("")
    axes[1].set_ylabel("")
    axes[1].tick_params(axis="x", labelsize=label_size)
    axes[1].tick_params(axis="y", labelsize=label_size)
    
    plt.show()

In [8]:
columns = data.columns.to_list()
columns = [column for column in columns if column not in ["stock_id", "time_id"]]

In [9]:
# for continuous_feature in columns:
#     visualize_continuous_feature(df_train, continuous_feature)